In [1]:
%%bash
echo "TODAY'S DATE:"
date
echo "------------"
echo ""
#Display operating system info
lsb_release -a
echo ""
echo "------------"
echo "HOSTNAME: "; hostname 
echo ""
echo "------------"
echo "Computer Specs:"
echo ""
lscpu
echo ""
echo "------------"
echo ""
echo "Memory Specs"
echo ""
free -mh

TODAY'S DATE:
Tue Dec 17 07:00:28 PST 2019
------------

LSB Version:	core-9.20170808ubuntu1-noarch:security-9.20170808ubuntu1-noarch
Distributor ID:	Ubuntu
Description:	Ubuntu 18.04.3 LTS
Release:	18.04
Codename:	bionic

------------
HOSTNAME: 
computer

------------
Computer Specs:

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              4
On-line CPU(s) list: 0-3
Thread(s) per core:  2
Core(s) per socket:  2
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               58
Model name:          Intel(R) Core(TM) i7-3517U CPU @ 1.90GHz
Stepping:            9
CPU MHz:             1795.794
CPU max MHz:         3000.0000
CPU min MHz:         800.0000
BogoMIPS:            4788.78
Virtualization:      VT-x
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            4096K
NUMA node0 CPU(s):   0-3
Flags:               fpu vme de pse

### Set variables¶
%env are best for bash

In [2]:
%env wd=/home/samb/analyses/20191214_olur_genome_feature_and_intron_splitting
wd="/home/samb/analyses/20191214_olur_genome_feature_and_intron_splitting"

# File download
%env downloads_list=Olurida_v081.fa.fai Olurida_v081_genome_snap02.all.renamed.putative_function.domain_added.gff Olurida_v081-20190709.exon.gff
%env rsync_owl=owl:/volume1/web/halfshell/genomic-databank/

# Input/output files
%env chrome_sizes=Olurida_v081.sizes.txt
%env fa_index=Olurida_v081.fa.fai
%env maker_gff=Olurida_v081_genome_snap02.all.renamed.putative_function.domain_added.gff
%env base_name=Olurida_v081-20190709
%env exon_gff=Olurida_v081-20190709.exon.gff3
%env exon_gff_sorted=Olurida_v081-20190709.exon.sorted.gff3
%env gene_gff=Olurida_v081-20190709.gene.gff3
%env gene_gff_sorted=Olurida_v081-20190709.gene.sorted.gff3

%env exon_comp_bed=Olurida_v081-20190709.exon.sorted.comp.bed
%env intron_bed=Olurida_v081-20190709.introns.bed
%env intergenic_bed=Olurida_v081-20190709.intergenic.bed

# Programs
%env bedtools_dir=/home/samb/programs/bedtools-2.29.0/bin
%env samtools=/home/samb/programs/samtools-1.10

env: wd=/home/sam/analyses/20191214_olur_genome_feature_and_intron_splitting
env: downloads_list=Olurida_v081.fa.fai Olurida_v081_genome_snap02.all.renamed.putative_function.domain_added.gff Olurida_v081-20190709.exon.gff
env: rsync_owl=owl:/volume1/web/halfshell/genomic-databank/
env: chrome_sizes=Olurida_v081.sizes.txt
env: fa_index=Olurida_v081.fa.fai
env: maker_gff=Olurida_v081_genome_snap02.all.renamed.putative_function.domain_added.gff
env: base_name=Olurida_v081-20190709
env: exon_gff=Olurida_v081-20190709.exon.gff3
env: exon_gff_sorted=Olurida_v081-20190709.exon.sorted.gff3
env: gene_gff=Olurida_v081-20190709.gene.gff3
env: gene_gff_sorted=Olurida_v081-20190709.gene.sorted.gff3
env: exon_comp_bed=Olurida_v081-20190709.exon.sorted.comp.bed
env: intron_bed=Olurida_v081-20190709.introns.bed
env: intergenic_bed=Olurida_v081-20190709.intergenic.bed
env: bedtools_dir=/home/samb/programs/bedtools-2.29.0/bin
env: samtools=/home/samb/programs/samtools-1.10


### Create necessary directories

In [3]:
%%bash
mkdir --parents ${wd}

mkdir: cannot create directory ‘/home/sam’: Permission denied


CalledProcessError: Command 'b'mkdir --parents ${wd}\n'' returned non-zero exit status 1.

In [ ]:
cd {wd}

### Download files

In [ ]:
%%bash

input_files_array=("${downloads_list}")

for file in "${input_files_array[@]}"
do
  rsync \
  --archive \
  --verbose \
  "${rsync_owl}${file}" \
  .
## Uncomment lines below if need to use wget for downloads
# wget --quiet \
# --no-check-certificate \
# https://owl.fish.washington.edu/halfshell/genomic-databank/${file}
done
echo ""
echo ""
echo "----------------------------------------------------------"
echo ""



ls -lh

### Separate features

In [ ]:
%%bash

# Create UTR array
utr_array=(five_prime_UTR three_prime_UTR)

echo "Here are the features and their counts in ${maker_gff}:"
awk 'NR>1 {print $3}' "${maker_gff}" | sort | uniq -c
echo ""
echo ""
echo "----------------------------------------------"


# Create GFFs
for feature in "${utr_array[@]}"
do
   output="${base_name}.${feature}.gff3"
   head -n 1 "${maker_gff}" >> "${output}"
   awk -v feature="$feature" '$3 == feature {print}' "${maker_gff}" \
   >> "${output}"
   echo ""
   echo ""
   echo "${output}"
   echo "----------------------------------------------"
   head -n 5 "${output}"
done

### Check exon and gene GFFs

In [ ]:

%%bash
echo ${exon_gff}
head ${exon_gff}
echo ""
echo "----------------------"
echo ""
echo ${gene_gff}
head ${gene_gff}

### Create chromosome sizes file

In [ ]:
%%bash
# Change LC_COLLATE to use C for proper sorting of file
export LC_COLLATE=C

cut -f1,2 ${genome_fasta}.fai | sort > ${chrome_sizes}

head ${chrome_sizes}

### Sort GFFs

In [ ]:
%%bash
gff_array=("${exon_gff}" "${gene_gff}")
sorted_gff_array=("${exon_gff_sorted}" "${gene_gff_sorted}")

for index in "${!gff_array[@]}"
do
  { awk 'NR<2 {print}' "${gff_array[index]}"
   awk 'NR>1 {print}' "${gff_array[index]}" | "${bedtools_dir}"/bedtools sort -i -
  } >> "${sorted_gff_array[index]}"
  
  # Check out sorted GFFs
  echo "Previewing ${sorted_gff_array[index]}:"
  echo ""
  head "${sorted_gff_array[index]}"
  echo ""
  echo "Confirming sort order of ${sorted_gff_array[index]}:"
  echo ""
  cut -f1 "${sorted_gff_array[index]}" | uniq
  echo ""
done

### Create intergenic BED file

In [ ]:
%%bash

"${bedtools_dir}"/complementBed \
-i "${gene_gff_sorted}" \
-g "${chrome_sizes}" \
> "${intergenic_bed}"

echo "Previewing ${intergenic_bed}:"
echo ""
head "${intergenic_bed}"

### Make intron BED file

In [ ]:
%%bash
"${bedtools_dir}"/complementBed \
-i "${exon_gff_sorted}" \
-g "${chrome_sizes}" \
> "${exon_comp_bed}"

echo "Previewing ${exon_comp_bed}:"
echo ""
head "${exon_comp_bed}"
echo ""
echo "---------------------"

"${bedtools_dir}"/intersectBed \
-a "${gene_gff_sorted}" \
-b "${exon_comp_bed}" \
| awk -v OFS='\t' '{print $1,$4,$5}' \
> "${intron_bed}"

echo ""
echo "Previewing ${intron_bed}:"
echo ""
head "${intron_bed}"

### Cleanup

In [ ]:
%%bash
rm "${maker_gff}" \
"${exon_comp_bed}" \
"${exon_gff}" \
"${exon_gff_sorted}" \
"${gene_gff}" \
"${gene_gff_sorted}" \
"${fa_index}"
ls -ltrh